# 🧠 Modelo de Clasificación de Clientes Turísticos

Este notebook entrena un modelo de Machine Learning usando **Amazon SageMaker + Scikit-Learn**,
para predecir la **categoría del cliente** (por ejemplo, VIP o Frecuente) a partir de sus características de viaje.

📦 **Fuente de datos:** Archivos CSV generados por Athena en el bucket `turismo-datalake-31102025/athena-results/2025/11/03/`

In [4]:
import boto3
import pandas as pd
import numpy as np
import sagemaker
from datetime import datetime

session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = 'turismo-datalake-31102025'
prefix = 'athena-results/Unsaved/2025/11/03/'

s3 = boto3.client('s3')
print('Configuración de SageMaker lista.')

Configuración de SageMaker lista.


## 1️⃣ Buscar el archivo CSV más reciente en la carpeta S3

In [5]:
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.csv')]
if not files:
    raise Exception('No se encontraron archivos CSV en la ruta especificada.')
latest_file = max(files, key=lambda x: s3.head_object(Bucket=bucket, Key=x)['LastModified'])
print(f' Archivo más reciente encontrado: {latest_file}')

 Archivo más reciente encontrado: athena-results/Unsaved/2025/11/03/c5e916ee-733a-48b4-984d-375c1cd03887.csv


## 2️⃣ Cargar y explorar los datos

In [6]:
obj = s3.get_object(Bucket=bucket, Key=latest_file)
df = pd.read_csv(obj['Body'])
print(f'Dataset cargado con {len(df)} filas y {len(df.columns)} columnas.')
df.head()

Dataset cargado con 66639 filas y 10 columnas.


,cliente_id,nombre,apellido,tipo_viajero,total_viajes,lifetime_value,promedio_duracion_viaje,destinos_visitados,ultima_fecha_compra,categoria_cliente
0,12993,Reynaldo,Menéndez,Pareja,3,29461.941,12.333333,3,2025-08-15,VIP
1,12994,Camila,Barros,Familia,1,9011.170,10.000000,1,2025-07-21,VIP
2,12995,Gilberto,Ripoll,Familia,2,13962.970,7.500000,2,2024-10-06,VIP
3,12997,Noé,Blanch,Individual,2,24063.300,10.500000,2,2025-10-31,VIP
4,13000,Oriana,Codina,Grupo,4,36134.742,5.750000,4,2025-09-19,VIP


## 3️⃣ Preparar los datos para el modelo

In [ ]:
print(df.columns.tolist())
cols = ['tipo_viajero', 'total_viajes', 'lifetime_value', 'promedio_duracion_viaje', 'destinos_visitados', 'categoria_cliente']
df = df[[c for c in cols if c in df.columns]].copy()
df = df.dropna()
for c in ['total_viajes', 'lifetime_value', 'promedio_duracion_viaje', 'destinos_visitados']:
    df[c] = pd.to_numeric(df[c], errors='coerce')
df = pd.get_dummies(df, columns=['tipo_viajero'], drop_first=True)
print(f'Dataset listo para el entrenamiento con {df.shape[1]} columnas')
df.head()

['cliente_id', 'nombre', 'apellido', 'tipo_viajero', 'total_viajes', 'lifetime_value', 'promedio_duracion_viaje', 'destinos_visitados', 'ultima_fecha_compra', 'categoria_cliente']
Dataset listo para el entrenamiento con 8 columnas


,total_viajes,lifetime_value,promedio_duracion_viaje,destinos_visitados,categoria_cliente,tipo_viajero_Grupo,tipo_viajero_Individual,tipo_viajero_Pareja
0,3,29461.941,12.333333,3,VIP,False,False,True
1,1,9011.170,10.000000,1,VIP,False,False,False
2,2,13962.970,7.500000,2,VIP,False,False,False
3,2,24063.300,10.500000,2,VIP,False,True,False
4,4,36134.742,5.750000,4,VIP,True,False,False


## 4️⃣ Dividir en entrenamiento y prueba

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df.drop(columns=['categoria_cliente'])
y = df['categoria_cliente']
le = LabelEncoder()
y_encoded = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print('División completada:')
print('Entrenamiento:', X_train.shape)
print('Prueba:', X_test.shape)

División completada:
Entrenamiento: (53311, 7)
Prueba: (13328, 7)


## 5️⃣ Entrenar el modelo (Random Forest)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {acc:.2%}')
print('\nReporte de clasificación:')
print(classification_report(y_test, y_pred, target_names=le.classes_))

Precisión del modelo: 100.00%

Reporte de clasificación:
              precision    recall  f1-score   support

   Frecuente       1.00      1.00      1.00      1158
         VIP       1.00      1.00      1.00     12170

    accuracy                           1.00     13328
   macro avg       1.00      1.00      1.00     13328
weighted avg       1.00      1.00      1.00     13328



## 6️⃣ Guardar el modelo entrenado en S3

In [10]:
import joblib
import os

model_path = '/tmp/modelo_clientes.pkl'
joblib.dump((model, le), model_path)
s3_model_path = f's3://{bucket}/modelos/modelo_clientes_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl'
sagemaker.s3.S3Uploader.upload(model_path, s3_model_path)

print(f'Modelo guardado en S3: {s3_model_path}')

Modelo guardado en S3: s3://turismo-datalake-31102025/modelos/modelo_clientes_20251103_104844.pkl


## 7️⃣ Probar una predicción

In [14]:
sample = X_test.sample(1, random_state=1)
pred = model.predict(sample)
pred_label = le.inverse_transform(pred)[0]

print('Muestra:')
display(sample)
print(f'Predicción de categoría: {pred_label}')

Muestra:


,total_viajes,lifetime_value,promedio_duracion_viaje,destinos_visitados,tipo_viajero_Grupo,tipo_viajero_Individual,tipo_viajero_Pareja
18173,4,29154.43,7.75,4,False,True,False


Predicción de categoría: VIP
